In [52]:
import networkx as nx
import math
import random
import numpy as np
import copy
from datetime import datetime
from matplotlib import pyplot as plt
import import_ipynb
import GraphGenerator as gg

In [53]:
def move_obstacle(g, node, start_pos, goal_pos, obstacles, stuck_num):    
    ## segFault
    if stuck_num == 50:
        return False, node
    
    possible_moves = sorted([node for node in g[node] if node != goal_pos and node != start_pos])
    # #recursion
    if len(possible_moves) == 0:
        return False, node
    
    chosen_move = random.sample(possible_moves, k=1)[0]
    if(chosen_move in obstacles):
        #recursion
        return move_obstacle(g, chosen_move, start_pos, goal_pos, obstacles, stuck_num+1)
        
    
    obstacles.remove(node)
    obstacles.append(chosen_move)

    # print(f'Obstacle moves: {node} -> {chosen_move}')
    return True, chosen_move

In [54]:
def move_robot(g, u, obstacles):
    neighbors = [node for node in g[u] if node not in obstacles]
    if len(neighbors) == 0:
        return u

    chosen_move = random.sample(neighbors, k=1)[0]
    # print(f'Robot moves: {u} -> {chosen_move}')
    return chosen_move

In [1]:
def find_path(g, start_pos, goal_pos, obstacles, num_iterations):
    start_time = datetime.now()
    robot_move_prob = 0.8
    min_length = float('inf')
    min_iteration = float('inf')
    obstacles_start = copy.deepcopy(obstacles)

    best_moves = []
    length_per_iteration = []
    
    for iteration in range(num_iterations):
        u = start_pos
        robot_moves = 0
        obstacle_moves = 0
        moves = []
        while (u != goal_pos):
            if (random.random() > robot_move_prob):
                next_node = move_robot(g, u, obstacles)
                if next_node == u:
                    u = start_pos
                    robot_moves = 0
                    continue
                moves.append(('r', f'{u}->{next_node}'))
                u = next_node
                robot_moves += 1
                
            else:
                random_obstacle = random.sample(obstacles, k=1)[0]
                # obstacle_moves += 1 if move_obstacle(g, random_obstacle, start_pos, goal_pos, obstacles, stuck_num=0) else 0
                obstacle_moved, moved_to = move_obstacle(g, random_obstacle, start_pos, goal_pos, obstacles, stuck_num=0)
                if obstacle_moved:
                    moves.append(('o', f'{random_obstacle}->{moved_to}'))
                    obstacle_moves += 1
    
        num_of_steps = robot_moves + obstacle_moves
        length_per_iteration.append(num_of_steps)
        # print(f'Path length for iteration {iteration+1}: {num_of_steps}')
        # print(f'Best path of iteration {iteration}: {moves}')
        if num_of_steps < min_length:
            min_length = num_of_steps
            min_iteration = iteration + 1
            best_moves = copy.deepcopy(moves)

        obstacles = copy.deepcopy(obstacles_start)

    end_time = datetime.now()
    print(f'The minimum length of {min_length} was found in iteration {min_iteration}')
    print(f'Time it took to finish the search: {(end_time - start_time).total_seconds()}')
    print('Best path of all iterations: ')
    for i in range(len(best_moves)):
        print(best_moves[i])

    # plt.plot(range(1, num_iterations+1), length_per_iteration, color='xkcd:ruby', label='Best Weight')
    # plt.xlabel('Iterations')
    # plt.ylabel('Best Weight')
    # plt.title('Shortest Number of Steps in Each Iteration')
    # plt.legend()
    # plt.xticks(np.linspace(1, 100, 10))
    return length_per_iteration

In [56]:
# find_path(g, start_pos, goal_pos, obstacles, num_iterations=100)